In [244]:
import sys
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install rpy2


Processing /Users/ychen/Library/Caches/pip/wheels/76/03/bb/589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074/sklearn-0.0-py2.py3-none-any.whl
  Using cached scikit_learn-0.23.2-cp38-cp38-macosx_10_9_x86_64.whl (7.2 MB)
  Using cached threadpoolctl-2.1.0-py3-none-any.whl (12 kB)


In [247]:
# Minimum packages
import numpy as np
import pandas as pd
import re
import bisect
import os

import matplotlib.pyplot as plt
plt.style.use('classic')
%matplotlib inline
import seaborn as sns
sns.set()

from IPython.display import Image

from sklearn.linear_model import LogisticRegression

from rpy2.robjects.packages import importr

import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri
from rpy2.robjects import default_converter
from rpy2.robjects.conversion import localconverter


In [196]:
# import functions
from lib.amt_zeros_filter import amt_zeros_filter
from lib.amt_zeros_pct_filter import amt_zeros_pct_filter
from lib.whole_number_pct_filter import whole_number_pct_filter
from lib.exec_137_filter import exec_137_filter
from lib.exec_137_pct_filter import exec_137_pct_filter

from lib.is_name_factor import is_name_factor

from lib.compute_iv_numeric import compute_iv_numeric
from lib.iv_numeric_select import iv_numeric_select
from lib.compute_iv_cat import compute_iv_cat
from lib.iv_cat_select import iv_cat_select

from lib.log_transform import log_transform
from lib.recode import recode
from lib.create_dummy import create_dummy
from lib.recode_category_2_dummies import recode_category_2_dummies


print(amt_zeros_filter(123.00))
print(amt_zeros_filter(123.01))
print(amt_zeros_filter(123.1))
print(amt_zeros_filter(1230.01))
print(amt_zeros_filter(1230))


2
0
0
1
3


In [208]:
data_dir = '/Users/ychen/Documents/results/research/'
inline = 'lr'
mydate = '20200930'


In [14]:
# Training
xfer_84_90 = pd.read_csv(data_dir + "xfers8485845-9019963.csv", encoding='latin-1')
print(xfer_84_90.shape)
print(xfer_84_90.columns)

# Validation
xfer_90_95 = pd.read_csv(data_dir + "xfers9027199-9536535.csv", encoding='latin-1')
print(xfer_90_95.shape)


(483895, 93)
Index(['xferId', 'account_age_at_xfer_creation_day_continuous',
       'sender_age_at_xfer_creation_year_continuous',
       'send_local_time_hour_continuous',
       'send_local_time_day_of_week_continuous',
       'send_local_time_day_of_month_continuous',
       'send_local_time_day_of_year_continuous',
       'send_local_time_month_continuous', 'recipient_country_factor',
       'recipient_state_country_factor', 'send_amount_continuous',
       'send_amount_thousand_continuous', 'send_amount_hundred_continuous',
       'send_amount_ten_continuous', 'send_amount_one_continuous',
       'send_amount_cent_continuous', 'receive_amount_continuous',
       'receive_currency_factor', 'payment_type_factor',
       'receive_method_factor', 'geo_code_factor', 'network_id_factor',
       'expedited_factor', 'designated_marketing_area_factor',
       'routing_number_factor', 'account_last_four_digits_factor',
       'browser_string_length_in_characters_factor', 'browser_language_f

In [11]:
# Keep the 1st dup
trn = xfer_84_90.drop_duplicates(subset='xferId', keep='first')
print(trn.shape)

val = xfer_90_95.drop_duplicates(subset='xferId', keep='first')
print(val.shape)


(483895, 93)
(466615, 93)


In [15]:
# Create numeric fraud flag
trn = trn.assign(flag = [0 if pd.isnull(x) else 1 for x in trn.fraud_flagged_after_pass_factor])
print(trn.shape)
print(trn.flag.value_counts())
print(trn.flag.mean())

val = val.assign(flag = [0 if pd.isnull(x) else 1 for x in val.fraud_flagged_after_pass_factor])
print(val.shape)
print(val.flag.value_counts())
print(val.flag.mean())


(483895, 94)
0    480211
1      3684
Name: flag, dtype: int64
0.007613221876646793
(466615, 94)
0    464342
1      2273
Name: flag, dtype: int64
0.004871253603077484


In [19]:
# Move flag to the 1st column
trn = trn[ ['flag'] + [ col for col in trn.columns if col != 'flag' ] ]
val = val[ ['flag'] + [ col for col in val.columns if col != 'flag' ] ]

print(trn.columns)
print(trn.shape)
print(val.shape)


Index(['flag', 'xferId', 'account_age_at_xfer_creation_day_continuous',
       'sender_age_at_xfer_creation_year_continuous',
       'send_local_time_hour_continuous',
       'send_local_time_day_of_week_continuous',
       'send_local_time_day_of_month_continuous',
       'send_local_time_day_of_year_continuous',
       'send_local_time_month_continuous', 'recipient_country_factor',
       'recipient_state_country_factor', 'send_amount_continuous',
       'send_amount_thousand_continuous', 'send_amount_hundred_continuous',
       'send_amount_ten_continuous', 'send_amount_one_continuous',
       'send_amount_cent_continuous', 'receive_amount_continuous',
       'receive_currency_factor', 'payment_type_factor',
       'receive_method_factor', 'geo_code_factor', 'network_id_factor',
       'expedited_factor', 'designated_marketing_area_factor',
       'routing_number_factor', 'account_last_four_digits_factor',
       'browser_string_length_in_characters_factor', 'browser_language_factor

In [56]:
# Derived payment amount signals
trn = trn.assign(amt_zeros = [amt_zeros_filter(x) for x in trn.send_amount_continuous])
trn = trn.assign(amt_zeros_pct = [amt_zeros_pct_filter(x) for x in trn.send_amount_continuous])
trn = trn.assign(whole_number_pct = [whole_number_pct_filter(x) for x in trn.send_amount_continuous])
trn = trn.assign(filter_137 = [exec_137_filter(x) for x in trn.send_amount_continuous])
trn = trn.assign(filter_137_pct = [exec_137_pct_filter(x) for x in trn.send_amount_continuous])
print(trn[['amt_zeros', 'amt_zeros_pct', 'whole_number_pct', 'filter_137', 'filter_137_pct']].describe())

derived_vars = ['amt_zeros', 'amt_zeros_pct', 'whole_number_pct', 'filter_137', 'filter_137_pct']
print(trn[derived_vars].describe())


           amt_zeros  amt_zeros_pct  whole_number_pct     filter_137  \
count  483895.000000  483895.000000     483895.000000  483895.000000   
mean        3.120214       0.643769          0.248987       0.637301   
std         0.942460       0.179122          0.045217       0.659071   
min         0.000000       0.000000          0.000000       0.000000   
25%         3.000000       0.600000          0.250000       0.000000   
50%         3.000000       0.750000          0.250000       1.000000   
75%         4.000000       0.800000          0.250000       1.000000   
max         5.000000       0.833333          0.333333       5.000000   

       filter_137_pct  
count   483895.000000  
mean         0.127553  
std          0.131073  
min          0.000000  
25%          0.000000  
50%          0.200000  
75%          0.200000  
max          0.833333  
           amt_zeros  amt_zeros_pct  whole_number_pct     filter_137  \
count  483895.000000  483895.000000     483895.000000  483895.0

In [70]:
# Get factor signals
var_names = trn.columns
#print(var_names)
var_names_sub = list(set(var_names).difference(set(['flag', 'xferId'])))
#print(var_names_sub)

factor_list = [is_name_factor(x) for x in var_names_sub]
#print(factor_list)

# Only factor signals in the list.
factor_names = [var_names_sub[i] for i, val in enumerate(factor_list) if val]  
print(factor_names)


['xfer_in_pass_workflow_factor', 'recipient_last_name_only_has_normal_character_factor', 'account_last_four_digits_factor', 'recipient_country_is_gt_factor', 'geo_code_is_10_factor', 'geo_code_is_170_factor', 'recipient_country_is_pl_factor', 'browser_string_exists_factor', 'designated_marketing_area_factor', 'network_id_is_5180_factor', 'designated_marketing_area_is_623_factor', 'recipient_country_is_mx_factor', 'recipient_state_country_is_MX_MEX_factor', 'recipient_state_country_is_PH_CAV_factor', 'recipient_state_country_is_PH_CEB_factor', 'recipient_city_exists_factor', 'recipient_country_is_ph_factor', 'recipient_country_is_in_factor', 'geo_code_is_87_factor', 'recipient_state_country_is_PE_LIM_factor', 'browser_string_length_in_characters_is_147_factor', 'designated_marketing_area_is_656_factor', 'device_id_exists_factor', 'browser_language_is_en_US_en_q_factor', 'browser_string_length_in_characters_is_50_factor', 'browser_string_length_in_characters_factor', 'recipient_state_cou

In [108]:
# Get numeric signals.
num_names = list(set(var_names_sub).difference(set(factor_names)))
#print(num_names)

# Finalize both factor and numeric signals.
# Eyeball to find non-qualified numeric signals.
factor_to_add = ["send_local_time_hour_continuous", "send_local_time_day_of_week_continuous", 
                 "send_local_time_day_of_month_continuous", "send_local_time_day_of_year_continuous", 
                 "send_local_time_month_continuous"]
num_names_sub = list(set(num_names).difference(set(factor_to_add)))
print(num_names_sub)

# Eyeball # of categories for numeric signals, hand-pick too few categories. 
print([trn[x].nunique() for x in num_names_sub])

factor_tmp = ["browser_language_is_en_us_en_q_factor_1", "browser_language_is_en_us_factor_1"]
factor_to_add_more = factor_to_add + factor_tmp
#print(factor_to_add_more)

num_names_final = set(num_names_sub).difference(set(factor_tmp))
print(num_names_final)
print(len(num_names_final))

factor_names_final = factor_names + factor_to_add_more
print(len(factor_names_final))


['receive_amount_continuous', 'send_amount_one_continuous', 'amt_zeros_pct', 'sender_age_at_xfer_creation_year_continuous', 'send_amount_continuous', 'browser_language_is_en_us_en_q_factor_1', 'send_amount_cent_continuous', 'filter_137', 'send_amount_hundred_continuous', 'send_amount_ten_continuous', 'browser_language_is_en_us_factor_1', 'filter_137_pct', 'send_amount_thousand_continuous', 'whole_number_pct', 'amt_zeros', 'account_age_at_xfer_creation_day_continuous']
[68819, 10, 12, 84, 7251, 2, 100, 6, 10, 10, 2, 12, 3, 5, 6, 2282]
{'filter_137', 'send_amount_ten_continuous', 'filter_137_pct', 'send_amount_thousand_continuous', 'amt_zeros', 'account_age_at_xfer_creation_day_continuous', 'receive_amount_continuous', 'send_amount_one_continuous', 'amt_zeros_pct', 'sender_age_at_xfer_creation_year_continuous', 'send_amount_continuous', 'send_amount_cent_continuous', 'send_amount_hundred_continuous', 'whole_number_pct'}
14
83


In [109]:
# Rank signals by descending in IV (Information Values)
ivs = iv_numeric_select(trn.flag, trn[num_names_final], 1, 1, 20)
print(ivs)


0
0.008754692494601011
1
0.050449404602050496
2
0.01721551284249189
3
0.0015272858896430057
4
0.0890877228479092
5
0.9379678156233175
6
0.1661619730661164
7
0.06474093100810747
8
0.08194693628645457
9
0.049709351819644476
10
0.08311022356557828
11
0.0
12
0.03545640607120097
13
0.10039617486409529
                                        varname        iv
5   account_age_at_xfer_creation_day_continuous  0.937968
6                     receive_amount_continuous  0.166162
13                             whole_number_pct  0.100396
4                                     amt_zeros  0.089088
10                       send_amount_continuous  0.083110
8                                 amt_zeros_pct  0.081947
7                    send_amount_one_continuous  0.064741
1                    send_amount_ten_continuous  0.050449
9   sender_age_at_xfer_creation_year_continuous  0.049709
12               send_amount_hundred_continuous  0.035456
2                                filter_137_pct  0.017216
0     

In [111]:
# Make backups
trn.to_csv(data_dir + 'trn_' + inline + '_' + mydate + '.csv', index=False)
val.to_csv(data_dir + 'val_' + inline + '_' + mydate + '.csv', index=False)
ivs.to_csv(data_dir + 'ivs_numeric_' + inline + '_' + mydate + '.csv', index=False)


In [117]:
# Extracted numeric signals whose iv >= 0.01
ivs_final = ivs[ivs.iv > 0.01]
print(ivs_final)

# Numeric signals identified so far
iv_num_names = ivs_final.varname
print(iv_num_names)

# Calculate reverse sampling weights for training negatives/positives
# No subsampling therefore all reverse sampling weights are 1
w_trn_neg = 1
w_trn_pos = 1
w_val_neg = 1
w_val_pos = 1


                                        varname        iv
5   account_age_at_xfer_creation_day_continuous  0.937968
6                     receive_amount_continuous  0.166162
13                             whole_number_pct  0.100396
4                                     amt_zeros  0.089088
10                       send_amount_continuous  0.083110
8                                 amt_zeros_pct  0.081947
7                    send_amount_one_continuous  0.064741
1                    send_amount_ten_continuous  0.050449
9   sender_age_at_xfer_creation_year_continuous  0.049709
12               send_amount_hundred_continuous  0.035456
2                                filter_137_pct  0.017216
5     account_age_at_xfer_creation_day_continuous
6                       receive_amount_continuous
13                               whole_number_pct
4                                       amt_zeros
10                         send_amount_continuous
8                                   amt_zeros_pct
7   

In [137]:
# Eyeball # of categories for categorical signals 
n_cats = [trn[x].nunique() for x in factor_names_final]
print(n_cats)
print(factor_names_final[:10])

# Remove those > 25 categories
filt_factor_names_final = [factor_names_final[i] for i, val in enumerate(n_cats) if ((val>=2) and (val<=25))]
print(len(filt_factor_names_final))
print(filt_factor_names_final[:10])


[1, 2, 10000, 2, 2, 2, 2, 2, 208, 2, 2, 2, 2, 2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 219, 2, 2, 2, 2, 1, 2, 2, 588, 2, 2, 2, 2, 2, 128, 1, 2, 2, 2, 84, 2, 2, 2, 2, 692, 22, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 32, 2, 2, 2, 2, 2, 2, 1, 2, 9139, 2, 2, 2, 2, 24, 7, 31, 62, 3, 2, 2]
['xfer_in_pass_workflow_factor', 'recipient_last_name_only_has_normal_character_factor', 'account_last_four_digits_factor', 'recipient_country_is_gt_factor', 'geo_code_is_10_factor', 'geo_code_is_170_factor', 'recipient_country_is_pl_factor', 'browser_string_exists_factor', 'designated_marketing_area_factor', 'network_id_is_5180_factor']
66
['recipient_last_name_only_has_normal_character_factor', 'recipient_country_is_gt_factor', 'geo_code_is_10_factor', 'geo_code_is_170_factor', 'recipient_country_is_pl_factor', 'browser_string_exists_factor', 'network_id_is_5180_factor', 'designated_marketing_area_is_623_factor', 'recipient_country_is_mx_factor', 'recipient_state_country_is_MX_MEX_factor']


In [138]:
# IV for categorical signals.
civs = iv_cat_select(trn.flag, trn[filt_factor_names_final], 1, 1)
print(civs)


0
0.0002170624242706866
1
0.08703651239603588
2
0.22834510650883982
3
0.07259584623351785
4
0.001061801153674986
5
0.020337721135539978
6
0.02255657551409101
7
0.0003660801945171507
8
0.021172623905302126
9
0.0016539235010264598
10
0.0020244621653595137
11
0.001944008596097968
12
0.0008713453087032076
13
0.007631838043532009
14
0.020383739506079995
15
0.0006279012749610888
16


<ipython-input-124-999bdd5789f2>:195: RuntimeWarning: divide by zero encountered in true_divide
  wpattern = np.log(prgoods/prbads)


3.523338896634849e-07
17
0.021455581123861932
18
0.0008937509930315356
19
2.6069030326110356e-06
20
0.0034910217603944978
21
0.037219260176742895
22
0.0006451035247879095
23
0.005628250093321927
24
0.007808397615464144
25
7.105810446332323e-08
26
0.3202327440034414
27
0.5449996431824504
28
0.00840896983089165
29
0.20379714671614482
30
0.002355459345922203
31
0.00032664290549315364
32
3.130470404856642e-05
33
0.0002795805585224838
34
0.13030391151898293
35
4.107973206190968e-05
36
0.0006734192882319411
37
0.002234268132805953
38
0.14627130591278245
39
0.00029019617425553974
40
0.17609160814726565
41
7.664607745586282e-05
42
0.004416323593635568
43
0.04792919256359186
44
0.02581160829157074
45
3.914046349580667e-08
46
0.001153835307846309
47
0.0033777331366117392
48
0.0005828470117011795
49
3.914046349580667e-08
50
0.0005925390805805891
51
0.0032062926187605974
52
0.00344920254254306
53
5.599326133229275e-05
54
2.8265709443220124e-05
55
0.02416869231354018
56
0.00021070704242650648
57
0.

In [139]:
# Continue make backup
civs.to_csv(data_dir + 'civs_numeric_' + inline + '_' + mydate + '.csv', index=False)


In [146]:
# Extracted categorical signals whose iv >= 0.01
civs_final = civs[civs.iv > 0.01]
#print(len(civs_final))
#print(civs_final)

# Categorical signals identified so far
iv_fac_names = civs_final.varname
print(iv_fac_names)
print(len(iv_fac_names))

# Eyeball # of categories again
print([trn[x].nunique() for x in iv_fac_names])


27                                    expedited_factor
26                     recipient_address_exists_factor
57                                is_first_xfer_factor
2                                geo_code_is_10_factor
29                           network_id_is_5020_factor
40                               receive_method_factor
38                             receive_currency_factor
34            recipient_state_country_is_PH_MAN_factor
65                  browser_language_is_en_us_factor_1
1                       recipient_country_is_gt_factor
64             browser_language_is_en_us_en_q_factor_1
3                               geo_code_is_170_factor
61                     send_local_time_hour_continuous
43                               geo_code_is_33_factor
21                                 payment_type_factor
44                      recipient_country_is_pe_factor
55    browser_string_length_in_characters_is_90_factor
6                            network_id_is_5180_factor
17        

In [150]:
# Look at numeric signals value ranges
# Check if need for clammping
# Check if need for NA handling
print(trn.shape)
print(trn[iv_num_names].describe())

# After eyeballing, no need for clamping, nor NA handling.


(483895, 99)
       account_age_at_xfer_creation_day_continuous  receive_amount_continuous  \
count                                483895.000000               4.838950e+05   
mean                                    539.433668               4.503084e+04   
std                                     476.116320               2.358342e+05   
min                                       0.000000               6.000000e-01   
25%                                     165.000000               6.000000e+02   
50%                                     425.000000               3.128500e+03   
75%                                     776.000000               1.050900e+04   
max                                    2454.000000               1.473509e+07   

       whole_number_pct      amt_zeros  send_amount_continuous  amt_zeros_pct  \
count     483895.000000  483895.000000           483895.000000  483895.000000   
mean           0.248987       3.120214              323.793398       0.643769   
std           

In [158]:
# Log transformation for $ amount
lvar_names = ["receive_amount_continuous", "send_amount_continuous", "account_age_at_xfer_creation_day_continuous"]
to_log_df = trn[lvar_names]
print(to_log_df.describe())

log_df = log_transform(to_log_df)
print(log_df.describe())



       receive_amount_continuous  send_amount_continuous  \
count               4.838950e+05           483895.000000   
mean                4.503084e+04              323.793398   
std                 2.358342e+05              456.878301   
min                 6.000000e-01                1.000000   
25%                 6.000000e+02              100.000000   
50%                 3.128500e+03              165.000000   
75%                 1.050900e+04              340.000000   
max                 1.473509e+07             2999.000000   

       account_age_at_xfer_creation_day_continuous  
count                                483895.000000  
mean                                    539.433668  
std                                     476.116320  
min                                       0.000000  
25%                                     165.000000  
50%                                     425.000000  
75%                                     776.000000  
max                                

/Users/ychen/Documents/projects/research/env3/lib/python3.8/site-packages/pandas/core/series.py:679: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


       ln_receive_amount_continuous  ln_send_amount_continuous  \
count                 483895.000000              483895.000000   
mean                       7.968666                   5.215556   
std                        2.235243                   0.998559   
min                       -0.510826                   0.000000   
25%                        6.396930                   4.605170   
50%                        8.048309                   5.105945   
75%                        9.259987                   5.828946   
max                       16.505742                   8.006034   

       ln_account_age_at_xfer_creation_day_continuous  
count                                   483895.000000  
mean                                         5.592899  
std                                          1.675955  
min                                          0.000000  
25%                                          5.105945  
50%                                          6.052089  
75%          

In [164]:
# Put all numeric signals together
fnum_names = set(iv_num_names).difference(set(lvar_names))
print(fnum_names)

trn_num = pd.concat([trn[fnum_names], log_df], axis=1)
print(trn_num.describe())

fnum_names_next = trn_num.columns
print(fnum_names_next)
print(len(fnum_names_next))


{'send_amount_one_continuous', 'amt_zeros_pct', 'sender_age_at_xfer_creation_year_continuous', 'send_amount_hundred_continuous', 'send_amount_ten_continuous', 'filter_137_pct', 'whole_number_pct', 'amt_zeros'}
       send_amount_one_continuous  amt_zeros_pct  \
count               483895.000000  483895.000000   
mean                     1.135269       0.643769   
std                      2.246095       0.179122   
min                      0.000000       0.000000   
25%                      0.000000       0.600000   
50%                      0.000000       0.750000   
75%                      0.000000       0.800000   
max                      9.000000       0.833333   

       sender_age_at_xfer_creation_year_continuous  \
count                                483895.000000   
mean                                     39.791399   
std                                      11.020512   
min                                      13.000000   
25%                                      31.000000 

In [235]:
# Recodes categorical signal by keeping top cateogries,
# which made up of cumulative 80% of total data points.
def recode_category_2_dummies(cat_signal, cat_signal_name, cut_off=0.8, default_cat='OTHER_'):
    """
    Args:
        cat_signal: An array of receipient country, a categorical signal.
        cat_signal_name: A string.
        cut_off: Cutoff for cumulative % of categories.
        default_cat: Default category name for unwanted categories.
    Returns:
        A design matrix of recoded category.
    """
    import pandas as pd
    import numpy as np

    # Recodes a categorical signal by reducing cateogries.
    def recode(e, elist, default='OTHER'):
        """
        Args:
            e: A category for testing.
            elist: A list of categories for keeping.
            default: A default category for coding unwanted categories.
        Returns:
            Recoded signal depending on if e in elist.
        """
        e = str(e)
        return e if e in elist else default

    # Get a design matrix of dummy signals.
    # Drop 1st column if it's binary signals.
    def create_dummy(x):
        """
        Args:
            x: An array of character or factor.
        Returns:
            A design matrix of dummy siganls.
        """
        s = pd.Series(x)
        y = pd.get_dummies(s)
        z = y.shape
        if z[1] == 2:
            # Drop 1st column if it's a binary signal.
            y = y.drop(y.columns[0], axis=1)
        return y

    x = pd.Series(cat_signal).value_counts().reset_index()
    x.columns = ['category', 'count']
    x = x.sort_values(by='count', ascending=False)
    x['tot'] = 1.0 * x['count'] / x['count'].sum()
    x['cumtot'] = np.cumsum(x['tot'])
    names = [x.category[i] for i, val in enumerate(x.cumtot) if val<=cut_off]
    z = [recode(x, names, default_cat) for x in cat_signal]
    df = create_dummy(z)
    # Prefix column names with '_'
    df.columns = [(cat_signal_name + '__' + n) for n in df.columns]
    return df


# Recode and create dummy matrix for "receive_currency_factor"
print(trn.receive_currency_factor.value_counts())

y = recode_category_2_dummies(trn.receive_currency_factor, "receive_currency_factor", 0.85, 'OTHER_')
print(y.shape)
print(y.describe())



PHP    183507
USD    109531
MXN     82282
COP     27233
INR     20956
DOP     13291
GTQ      8493
BRL      7529
HNL      7104
KRW      5923
GBP      5065
JMD      3743
EUR      3290
ZAR      1428
CAD      1278
AUD       877
ARS       696
UYU       593
PLN       540
PYG       317
BOB       217
PEN         2
Name: receive_currency_factor, dtype: int64
(483895, 5)
       receive_currency_factor__COP  receive_currency_factor__MXN  \
count                 483895.000000                 483895.000000   
mean                       0.056279                      0.170041   
std                        0.230459                      0.375669   
min                        0.000000                      0.000000   
25%                        0.000000                      0.000000   
50%                        0.000000                      0.000000   
75%                        0.000000                      0.000000   
max                        1.000000                      1.000000   

       receive

In [191]:
# Factors to dummies
fac_to_dummy = ["receive_currency_factor", "send_local_time_hour_continuous", "send_local_time_day_of_month_continuous"]

# Concate all dummy data frames
# First create an Empty DataFrame object 
dummy_df = pd.DataFrame()
for col in fac_to_dummy:
    print(col)
    trn[col] = trn[col].astype(str)
    y = recode_category_2_dummies(trn[col], col, 0.9, 'OTHER_')
    print(y.shape)
    dummy_df = pd.concat([dummy_df, y], axis=1)

print(dummy_df.shape)
print(dummy_df.nunique())


receive_currency_factor
(483895, 6)
send_local_time_hour_continuous
(483895, 17)
send_local_time_day_of_month_continuous
(483895, 26)
(483895, 49)
receive_currency_factor__COP                       2
receive_currency_factor__INR                       2
receive_currency_factor__MXN                       2
receive_currency_factor__OTHER_                    2
receive_currency_factor__PHP                       2
receive_currency_factor__USD                       2
send_local_time_hour_continuous__10                2
send_local_time_hour_continuous__11                2
send_local_time_hour_continuous__12                2
send_local_time_hour_continuous__13                2
send_local_time_hour_continuous__14                2
send_local_time_hour_continuous__15                2
send_local_time_hour_continuous__16                2
send_local_time_hour_continuous__17                2
send_local_time_hour_continuous__18                2
send_local_time_hour_continuous__19                2
send_

In [202]:
# To compute IV (Information values) for categorical singal with weighting.
# To model bads/frauds as 1.
# Similar to FICO scores.
# The lower the score, the higher the probability of bads.
# Hence the scores sorted in ascending order.
def compute_iv_cat(target, output, rs_rate_0=1, rs_rate_1=1):
    """
    Args:
        target: Array of target, 0 or 1 for binary classification.
        output: Array of categorical signal.
        rs_rate_0: Reverse sampling rate for negative, default is 1.
        rs_rate_1: Reverse sampling rate for positive, default is 1.
    Returns:
        Sum of information values across all categories.

    Usage example:
        ivs = ComputeIVCat(target, actual, 1, 1)
    """
    import pandas as pd
    import numpy as np

    a = pd.Series(output).unique()

    n = len(a)
    tot = np.repeat(0, n)
    bads = np.repeat(0, n)
    goods = np.repeat(0, n)
    tmp = [target[i] for i, val in enumerate(target) if val==0]
    ngoods = len(tmp)*rs_rate_0
    tmp = [target[i] for i, val in enumerate(target) if val==1]
    nbads = len(tmp)*rs_rate_1
    ntot = ngoods + nbads

    for i in np.arange(0, n):
        st = [target[ix] for ix, val in enumerate(output) if val==a[i]]
        tmp = [st[i] for i, val in enumerate(st) if val==0]
        goods[i] = len(tmp)*rs_rate_0
        tmp = [st[i] for i, val in enumerate(st) if val==1]
        bads[i] = len(tmp)*rs_rate_1
        tot[i] = goods[i] + bads[i]

    idx = np.argsort(a)
    a = a[idx]
    tot = tot[idx]
    bads = bads[idx]
    goods = goods[idx]

    cumtot = np.cumsum(tot, dtype=float)
    cumbads = np.cumsum(bads, dtype=float)
    cumgoods = np.cumsum(goods, dtype=float)
    pctpop = cumtot/ntot

    prgoods = goods/ngoods
    prbads = bads/nbads
    wpattern = np.log(prgoods/prbads)
    prcumgoods = np.cumsum(prgoods)
    prcumbads = np.cumsum(prbads)
    iv = (prgoods - prbads)*wpattern

    iv = [iv[i] for i, val in enumerate(iv) if np.isfinite(val)]
    iv = [iv[i] for i, val in enumerate(iv) if not np.isnan(val)]
    y = sum(iv)

    return y


# Calculate IV (Information Values) for categorical signal selection.
def iv_cat_select(flag, df, rs_rate_0=1, rs_rate_1=1):
    """
    Args:
        flag: The response (target).
        df: Data frame or matrix, all categorical signals
        rs_rate_0: Reverse sampling rate for negative, default is 1.
        rs_rate_1: Reverse sampling rate for positive, default is 1.
    Returns:
        Information values for each singal in descending order
    """
    import pandas as pd
    import numpy as np
    from compute_iv_cat import compute_iv_cat

    cols = df.columns
    ncol = len(cols)
    ivs = np.repeat(0.1, ncol)

    for i, val in enumerate(cols):
        print(i)
        signal = df[val]
        iv = compute_iv_cat(flag, signal, rs_rate_0, rs_rate_1)
        print(iv)
        ivs[i] = iv

    z = pd.DataFrame({"varname": cols, "iv": ivs})
    z = z.sort_values(by='iv', ascending=False)
    return z


# IV for dummy signals.
divs = iv_cat_select(trn.flag, dummy_df, 1, 1)
print(divs)


0
0.00840896983089165
1
0.007631838043532009
2
0.021172623905302126
3
0.011863459186126522
4
0.04967991282582912
5
0.024628419579638752
6
0.0031905534616502337
7
0.001816736103439965
8
1.0227191615233771e-06
9
5.076550884324546e-05
10
0.0006570609021847907
11
0.00042587403084194295
12
0.0004713978340211152
13
0.0009856555147495965
14
3.974426436569518e-05
15
0.00043467923606833897
16
0.00012320777037370302
17
0.00014646830411051953
18
0.0011449311488621185
19
0.0002795091060913594
20
0.009876610634971363
21
0.005227657247989419
22
0.023925770279749497
23
9.775214861844114e-06
24
0.000842983911692622
25
0.00032348953893402403
26
0.0001011743116931779
27
4.226844477961352e-05
28
1.4245215580385473e-05
29
0.004066645480469525
30
0.00011145980993797802
31
0.00019170886175291055
32
1.5428741924569034e-05
33
0.0015304875397080665
34
0.001812113631874183
35
2.8314756905659437e-08
36
0.0001154200022410851
37
0.0002418765275306496
38
0.0010485499942015756
39
0.0005881462368597325
40
0.001731589

In [213]:
# Extracted dummy signals whose iv >= 0.01
divs_final = divs[divs.iv > 0.01]
print(len(divs_final))
print(divs_final)

# Dummy signals identified so far
iv_dummy_names = divs_final.varname
print(iv_dummy_names)

trn_dummy = dummy_df[iv_dummy_names]
print(trn_dummy.shape)
print(trn_dummy.describe())

# Continue make backup
divs_final.to_csv(data_dir + 'divs_final_dummy_' + inline + '_' + mydate + '.csv', index=False)
trn_dummy.to_csv(data_dir + 'trn_dummy_' + inline + '_' + mydate + '.csv', index=False)


5
                                    varname        iv
4              receive_currency_factor__PHP  0.049680
5              receive_currency_factor__USD  0.024628
22  send_local_time_hour_continuous__OTHER_  0.023926
2              receive_currency_factor__MXN  0.021173
3           receive_currency_factor__OTHER_  0.011863
4                receive_currency_factor__PHP
5                receive_currency_factor__USD
22    send_local_time_hour_continuous__OTHER_
2                receive_currency_factor__MXN
3             receive_currency_factor__OTHER_
Name: varname, dtype: object
(483895, 5)
       receive_currency_factor__PHP  receive_currency_factor__USD  \
count                 483895.000000                 483895.000000   
mean                       0.379229                      0.226353   
std                        0.485196                      0.418471   
min                        0.000000                      0.000000   
25%                        0.000000                      0

In [216]:
##fac_to_dummy = ["receive_currency_factor", "send_local_time_hour_continuous", "send_local_time_day_of_month_continuous"]
print(fac_to_dummy)

print(len(iv_fac_names))
print(iv_fac_names)

final_fac_names = set(iv_fac_names).difference(set(fac_to_dummy))
print(len(final_fac_names))
print(final_fac_names)

# Not overlapping with numeric signals.  No more need to be done.
print(len(fnum_names_next))
final_num_names = set(fnum_names_next).difference(set(fac_to_dummy))
print(len(final_num_names))
print(final_num_names)


22
27                                    expedited_factor
26                     recipient_address_exists_factor
57                                is_first_xfer_factor
2                                geo_code_is_10_factor
29                           network_id_is_5020_factor
40                               receive_method_factor
38                             receive_currency_factor
34            recipient_state_country_is_PH_MAN_factor
65                  browser_language_is_en_us_factor_1
1                       recipient_country_is_gt_factor
64             browser_language_is_en_us_en_q_factor_1
3                               geo_code_is_170_factor
61                     send_local_time_hour_continuous
43                               geo_code_is_33_factor
21                                 payment_type_factor
44                      recipient_country_is_pe_factor
55    browser_string_length_in_characters_is_90_factor
6                            network_id_is_5180_factor
17     

In [236]:
# Consolidate factor signals.
trn_fac = trn[final_fac_names]
print(trn_fac.shape)
#print(trn_fac.nunique())

# Convert all factors to dummies, since Python regression doesn't auto take care of factor signals.
# Concate all dummy data frames
# First create an Empty DataFrame object 
fac_dummy_df = pd.DataFrame()
for col in final_fac_names:
    print(col)
    trn[col] = trn[col].astype(str)
    #y = recode_category_2_dummies_no_reduce(trn[col], col, 'OTHER_')
    y = recode_category_2_dummies(trn[col], col, 1.0, 'OTHER_')
    print(y.shape)
    fac_dummy_df = pd.concat([fac_dummy_df, y], axis=1)

print(fac_dummy_df.shape)
print(fac_dummy_df.nunique())


(483895, 20)
geo_code_is_87_factor
(483895, 1)
recipient_address_exists_factor
(483895, 1)
geo_code_is_33_factor
(483895, 1)
expedited_factor
(483895, 1)
recipient_country_is_gt_factor
(483895, 1)
recipient_country_is_pe_factor
(483895, 1)
device_id_exists_factor
(483895, 1)
network_id_is_5020_factor
(483895, 1)
geo_code_is_10_factor
(483895, 1)
recipient_state_country_is_PH_MAN_factor
(483895, 1)
browser_string_length_in_characters_is_90_factor
(483895, 1)
browser_language_is_en_us_en_q_factor_1
(483895, 1)
browser_string_exists_factor
(483895, 1)
payment_type_factor
(483895, 1)
is_first_xfer_factor
(483895, 1)
network_id_is_5180_factor
(483895, 1)
recipient_country_is_mx_factor
(483895, 1)
browser_language_is_en_us_factor_1
(483895, 1)
receive_method_factor
(483895, 3)
geo_code_is_170_factor
(483895, 1)
(483895, 22)
geo_code_is_87_factor__Y                               2
recipient_address_exists_factor__Y                     2
geo_code_is_33_factor__Y                               2

In [237]:
# IV for all factor dummy signals.
fivs = iv_cat_select(trn.flag, fac_dummy_df, 1, 1)
print(fivs)

# Extracted dummy signals whose iv >= 0.01
fivs_final = fivs[fivs.iv > 0.01]
print(len(fivs_final))
print(fivs_final)

# Dummy signals identified so far
fivs_final_names = fivs_final.varname
print(fivs_final_names)

fivs_final_df = fac_dummy_df[fivs_final_names]
print(fivs_final_df.shape)


0
0.020383739506079995
1
0.3202327440034414
2
0.04792919256359186
3
0.5449996431824504
4
0.08703651239603588
5
0.02581160829157074
6
0.021455581123861932
7
0.20379714671614482
8
0.22834510650883982
9
0.13030391151898293
10
0.02416869231354018
11
0.07280785578865914
12
0.020337721135539978
13
0.037219260176742895
14
0.3194548266004753
15
0.02255657551409101
16
0.021172623905302126
17
0.08970771928974203
18
0.1756486418644671
19
0.019173851998823377
20
0.14266597722546204
21
0.07259584623351785
                                              varname        iv
3                                 expedited_factor__1  0.545000
1                  recipient_address_exists_factor__Y  0.320233
14                            is_first_xfer_factor__1  0.319455
8                            geo_code_is_10_factor__Y  0.228345
7                        network_id_is_5020_factor__Y  0.203797
18                           receive_method_factor__1  0.175649
20                           receive_method_factor__4 

In [238]:
# Put all signals together
# Either numeric or dummy signals - no factor
trn_next = pd.concat([trn['flag'], trn_num, trn_dummy, fivs_final_df], axis=1)
print(trn_next.shape)
print(trn_next.nunique())

# Continue make backup
trn_next.to_csv(data_dir + 'trn_next_' + inline + '_' + mydate + '.csv', index=False)


(483895, 39)
flag                                                       2
send_amount_one_continuous                                10
amt_zeros_pct                                             12
sender_age_at_xfer_creation_year_continuous               84
send_amount_hundred_continuous                            10
send_amount_ten_continuous                                10
filter_137_pct                                            12
whole_number_pct                                           5
amt_zeros                                                  6
ln_receive_amount_continuous                           68819
ln_send_amount_continuous                               7251
ln_account_age_at_xfer_creation_day_continuous          2281
receive_currency_factor__PHP                               2
receive_currency_factor__USD                               2
send_local_time_hour_continuous__OTHER_                    2
receive_currency_factor__MXN                               2
receive_cur